# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# My Tutor: Evolved

## Imports & Setup

In [25]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

from openai import OpenAI
import ollama
import gradio as gr

# handling images
import base64
from io import BytesIO
from PIL import Image

# handling audio
from pydub import AudioSegment
from pydub.playback import play
from pathlib import Path

## Constants

In [2]:
MODEL_GPT = "gpt-4o-mini"

OLLAMA_API = "http://localhost:11434/api/chat"
LL_HEADERS = {"Content-Type": "application/json"}
MODEL_LLAMA = "llama3.2"

## Environment & Initialization

In [3]:
load_dotenv(override=True)

# OpenAI
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI key not set")

openai = OpenAI()

OpenAI key exists and begins with sk-proj-


## Prompts

In [8]:
system_prompt = "You are an assistant that takes technical questions and responds with an explanation, like a tutor. You should gently encourage the student to ask a question."

In [ ]:
def user_prompt_question(question):
    user_prompt = "You are a tutor assistant. Please provide an answer and explanation. Respond in Markdown. \nThe question is as follows: \n\n"
    user_prompt += question
    return user_prompt

In [6]:
example_question = """
Please explain what this code does and why:
yeild from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
print(user_prompt_question(example_question))

You are a tutor assistant. Please provide an answer and explanation. 
The question is as follows: 


Please explain what this code does and why:
yeild from {book.get("author") for book in books if book.get("author")}



## Chat Function w History

### OpenAI

In [9]:
def chat_gpt(message, history):

    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [10]:
gr.ChatInterface(fn=chat_gpt, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


### Llama 3.2

In [11]:
def chat_llama(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]

    stream = ollama.chat(model=MODEL_LLAMA, messages=messages, stream=True)

    response_text = ""
    for chunk in stream:
        response_text += chunk['message']['content']
        yield response_text


In [12]:
gr.ChatInterface(fn=chat_llama, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


In [33]:
def talker_gpt(message):
    # speech_file_path = Path(__file__).parent / "speech.mp3"
    with openai.audio.speech.with_streaming_response.create(
        model="tts-1",
        voice="alloy",
        input=message
    ) as response:
        audio_stream = BytesIO(response)
        response.stream_to_file(audio_stream)

        audio = AudioSegment.from_file(audio_stream, format="mp3")
        play(audio)

In [34]:
talker_gpt("How are you today?")

TypeError: a bytes-like object is required, not 'StreamedBinaryAPIResponse'

In [23]:
def chat_gpt(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        # talker_gpt(response)
        yield response
    
    # reply = response.choices[0].message.content
    # history += [{"role": "assistant", "content": reply}]

    # # talker_gpt(reply)

    # return history


In [24]:
gr.ChatInterface(fn=chat_gpt, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


In [ ]:
# # More involved Gradio code as we're not using the preset chat interface
# # Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately

# with gr.Blocks() as ui:
#     with gr.Row():
#         chatbot = gr.Chatbot(height=500, type="messages")
#     with gr.Row():
#         entry = gr.Textbox(label="Chat with your Tutor:")
#     with gr.Row():
#         clear = gr.Button("Clear")

    
#     def do_entry(message, history):
#         history += [{"role": "user", "content": message}]
#         return "", history
    
#     entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
#         chat_gpt, inputs=chatbot, outputs=[chatbot]
#     )
#     clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# ui.launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/home/ksg-dev/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ksg-dev/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ksg-dev/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ksg-dev/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1675, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ksg-dev/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 735,